In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, accuracy_score, auc
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, metrics, cv

# PDBP Analysis

In [2]:
path1=Path("/Users/zainabnazari/Desktop/pdbp/PPMI_PDBP_Log2CPM_sex_effect_removed_Norm_Quantile_together_PD_CTR")

In [3]:
pdbp_data=pd.read_csv(path1/"pdbp_data.csv",delimiter='\t')
pdbp_data

,ID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000285814,ENSG00000285843,ENSG00000285891,ENSG00000285913,ENSG00000285937,ENSG00000285946,ENSG00000285950,ENSG00000285975,Diagnosis,Sex
0,PDLK184WGE,-0.795371,-2.729347,4.132727,5.947212,3.915167,9.081336,2.544537,4.212770,5.115425,...,-2.786100,-2.942664,-2.770755,-2.770755,-2.788866,-2.887931,-2.825500,-2.781080,PD,M
1,PDXW222GG2,0.029172,-3.487062,5.383310,6.113187,4.208315,8.994625,4.462823,4.380455,5.551275,...,-3.431357,-3.268287,-3.452658,-3.452658,-3.427209,-3.317961,-3.380948,-3.442536,PD,F
2,PDDY096RP8,0.161952,-2.245680,4.764646,6.189538,4.310707,8.915631,3.514179,4.579706,5.807383,...,-3.480309,-3.362947,-3.490272,-3.490272,-3.479424,-3.199985,-3.393907,-3.483213,CTR,F
3,PDCJ149XWQ,-0.851656,-2.650620,5.398471,6.203822,4.209682,8.833031,3.559513,4.704305,5.508427,...,-2.730225,-2.912417,-2.710112,-2.710112,-2.733033,-2.850490,-2.775262,-2.722415,PD,M
4,PDYH176ZFA,-0.712838,-2.724630,4.980514,6.184612,4.256929,8.860026,2.689762,4.447423,5.452709,...,-2.783305,-2.940982,-2.767595,-2.767595,-2.785704,-2.886320,-2.823342,-2.777919,CTR,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,PDZA968YLU,-0.432641,-2.489323,4.779148,5.963757,4.270477,9.142878,2.490018,4.810756,5.216479,...,-2.593489,-2.857318,-2.565912,-2.565912,-2.597467,-2.775536,-2.670709,-2.583686,PD,M
1203,PDBE349TEN,-0.769956,-2.580791,4.700159,5.839763,3.898746,9.267990,4.262164,4.266996,5.253896,...,-2.674132,-2.891556,-2.649150,-2.649150,-2.678160,-2.821396,-2.735409,-2.664074,CTR,M
1204,PDCJ314NKT,-0.075972,-3.483480,4.797046,6.283134,4.454279,8.577536,3.152542,4.022217,5.745774,...,-3.416596,-3.247988,-3.442413,-3.442413,-3.414100,-3.301836,-3.365560,-3.424378,CTR,F
1205,PDYZ819YLQ,-1.151518,-2.418979,5.472407,6.263525,4.380882,8.987690,2.924825,4.543833,5.547137,...,-2.525174,-2.833294,-2.494825,-2.494825,-2.530494,-2.740900,-2.612083,-2.513835,PD,M


In [4]:
pdbp_data.set_index('ID', inplace=True)

In [5]:
X = pdbp_data.drop(['Diagnosis','Sex'], axis=1) 
y = pdbp_data['Diagnosis']

In [6]:
# Initialize CatBoost model
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=6, 
                           loss_function='Logloss', 
                           verbose=0)

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit model
model.fit(X, y)

# Get feature importances
feature_importances = model.get_feature_importance(Pool(X, label=y))

# Select features
selector = SelectFromModel(model, prefit=True, threshold='mean')
X_new = selector.transform(X)

# Get selected feature names
selected_features = X.columns[selector.get_support()]
print("Selected features:", selected_features)

# Create a new CatBoost model
model_selected = CatBoostClassifier(iterations=1000, 
                                    learning_rate=0.1, 
                                    depth=6, 
                                    loss_function='Logloss', 
                                    verbose=0)

# Cross-validation scores with selected features
scores = cross_val_score(model_selected, X_new, y, cv=cv, scoring='roc_auc')
print("Cross-validation AUC with selected features:", scores.mean())


/Users/zainabnazari/opt/miniconda3/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Selected features: Index(['ENSG00000002549', 'ENSG00000002746', 'ENSG00000004142',
       'ENSG00000004838', 'ENSG00000005001', 'ENSG00000005020',
       'ENSG00000005187', 'ENSG00000006007', 'ENSG00000006016',
       'ENSG00000007038',
       ...
       'ENSG00000284713', 'ENSG00000284808', 'ENSG00000284809',
       'ENSG00000284954', 'ENSG00000285155', 'ENSG00000285163',
       'ENSG00000285184', 'ENSG00000285210', 'ENSG00000285374',
       'ENSG00000285448'],
      dtype='object', length=2103)
Cross-validation AUC with selected features: 0.8177840375586853


 # PPMI Analysis

In [7]:
ppmi_data=pd.read_csv(path1/"ppmi_data.csv",delimiter='\t')
ppmi_data

,ID,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,...,ENSG00000285814,ENSG00000285843,ENSG00000285891,ENSG00000285913,ENSG00000285937,ENSG00000285946,ENSG00000285950,ENSG00000285975,Diagnosis,Sex
0,3000,-0.101375,-2.999518,4.328498,6.049779,4.211890,8.784764,2.173793,4.125101,5.434887,...,-3.478343,-3.364637,-3.487534,-3.487534,-3.477212,-3.160921,-3.276003,-3.480959,CTR,F
1,3001,-0.070030,-2.736090,5.246508,6.163900,4.088958,8.628968,3.412540,4.231814,5.556669,...,-2.791950,-2.947585,-2.776044,-2.776044,-2.794114,-2.892143,-2.832832,-2.785990,PD,M
2,3002,1.259571,-1.404902,5.358119,6.074396,4.424164,8.762913,2.727398,4.769609,5.340044,...,-3.481048,-3.323370,-3.490907,-3.490907,-3.479984,-2.779435,-2.944676,-3.483760,PD,F
3,3003,-1.071352,-2.890159,5.247370,6.124415,4.354913,8.363715,4.113420,4.505387,6.446975,...,-3.464768,-3.303630,-3.477475,-3.477475,-3.462194,-3.064134,-3.415254,-3.470379,PD,F
4,3004,-0.499520,-2.414768,5.351478,6.352868,4.315031,9.066409,4.021306,4.289712,5.111581,...,-3.333370,-3.078897,-3.322814,-3.322814,-3.333964,-2.935973,-2.932657,-3.329702,CTR,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,4102,0.394976,-3.020122,5.379363,6.271351,4.256049,8.136359,4.036716,4.581787,5.583952,...,-3.049144,-2.816837,-3.039936,-3.039936,-3.050273,-3.101747,-1.829699,-3.045297,PD,M
541,4108,-1.064643,-2.870947,4.891916,6.071242,4.519018,8.626706,3.253652,4.551400,6.067533,...,-3.111776,-3.190143,-3.105350,-3.105350,-3.113076,-3.159333,-3.128504,-3.108994,PD,M
542,4115,-0.626887,-2.947472,5.040913,6.138357,4.725125,8.664279,2.839370,4.178273,5.699001,...,-2.978779,-3.080310,-2.969678,-2.969678,-2.979836,-2.605039,-3.001129,-2.975235,PD,M
543,4136,-1.850876,-2.993623,4.164878,5.503493,3.792458,9.119153,2.870142,3.759730,6.305601,...,-3.265123,-3.207870,-3.373228,-3.373228,-3.386139,-3.196720,-3.413071,-3.380948,PD,M


In [8]:
X_ = ppmi_data.drop(['Diagnosis','Sex'], axis=1) 
y_ = ppmi_data['Diagnosis']

In [9]:
# Initialize CatBoost model
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=6, 
                           loss_function='Logloss', 
                           verbose=0)

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit model
model.fit(X_, y_)

# Get feature importances
feature_importances = model.get_feature_importance(Pool(X_, label=y_))

# Select features
selector = SelectFromModel(model, prefit=True, threshold='mean')
X_new = selector.transform(X_)

# Get selected feature names
selected_features_x = X_.columns[selector.get_support()]
print("Selected features:", selected_features_x)

# Create a new CatBoost model
model_selected = CatBoostClassifier(iterations=1000, 
                                    learning_rate=0.1, 
                                    depth=6, 
                                    loss_function='Logloss', 
                                    verbose=0)

# Cross-validation scores with selected features
scores = cross_val_score(model_selected, X_new, y_, cv=cv, scoring='roc_auc')
print("Cross-validation AUC with selected features:", scores.mean())

/Users/zainabnazari/opt/miniconda3/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Selected features: Index(['ENSG00000002330', 'ENSG00000003509', 'ENSG00000004660',
       'ENSG00000004799', 'ENSG00000005882', 'ENSG00000006025',
       'ENSG00000006128', 'ENSG00000006194', 'ENSG00000007062',
       'ENSG00000007080',
       ...
       'ENSG00000284646', 'ENSG00000284702', 'ENSG00000284713',
       'ENSG00000284739', 'ENSG00000284928', 'ENSG00000284966',
       'ENSG00000285016', 'ENSG00000285041', 'ENSG00000285077',
       'ENSG00000285374'],
      dtype='object', length=1530)
Cross-validation AUC with selected features: 0.8056565646447039


In [14]:
len(list(selected_features))

2103

In [15]:
len(list(selected_features_x))

1530

In [18]:
common_elements = list(set(selected_features) & set(selected_features_x))
#len(common_elements): 130


# Applying the common genes to the PPMI and PDBP

# PDBP filtered

In [23]:
new_X = X[common_elements]
new_X

,ENSG00000131686,ENSG00000229385,ENSG00000089041,ENSG00000185745,ENSG00000187546,ENSG00000119922,ENSG00000178750,ENSG00000237714,ENSG00000113100,ENSG00000166171,...,ENSG00000103154,ENSG00000270339,ENSG00000163961,ENSG00000138303,ENSG00000225146,ENSG00000254566,ENSG00000179104,ENSG00000254488,ENSG00000113318,ENSG00000161800
ID,,,,,,,,,,,,,,,,,,,,,
PDLK184WGE,-2.005897,-3.101419,4.708054,5.068295,-3.207330,7.630859,-2.438061,-2.957026,-3.289049,1.342890,...,1.348002,1.838823,5.828766,3.682767,-2.793712,-2.848653,5.529585,-3.494118,5.166472,3.398268
PDXW222GG2,-1.065682,-2.383741,4.744261,6.095587,-2.989527,8.700905,-3.113255,-3.254063,-0.701251,3.378604,...,-0.288630,2.341670,6.001804,3.162549,-3.420883,-3.355740,5.140661,-2.580014,4.753855,3.917537
PDDY096RP8,-1.710998,-2.583460,5.464808,6.423862,-1.422475,8.698356,-2.910790,-3.101074,-1.984013,2.621514,...,0.767118,-0.106666,5.969811,3.785279,-3.329331,-3.459151,5.945679,-3.415748,5.273969,3.595647
PDCJ149XWQ,-1.915031,-3.082453,4.907778,6.157344,-3.185503,8.430855,-1.808679,-2.928011,-1.642657,2.158757,...,1.102554,1.378953,5.872099,3.132372,-2.739639,-2.799915,4.942087,-2.597950,5.124382,3.253652
PDYH176ZFA,-0.952001,-3.101977,5.140661,7.377671,-3.207256,8.075091,-3.093223,-2.955486,-2.549490,1.030308,...,0.715560,1.731263,5.688729,3.111170,-2.278001,-2.846508,5.468403,-2.224202,5.496748,3.341665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PDZA968YLU,-2.444734,-3.054902,5.099600,6.401937,-3.165376,8.505935,-3.044824,-2.878132,-2.301773,2.663551,...,1.140658,2.158757,5.961224,3.437889,-2.606603,-2.711965,5.835937,-3.486956,5.448668,3.522950
PDBE349TEN,-1.270736,-3.074202,5.092419,7.719645,-3.181243,8.744322,-3.065660,-2.911304,-3.272119,2.313629,...,0.815746,2.546319,5.653981,3.271130,-2.102251,-2.766799,5.326087,-2.048961,4.988089,3.273860
PDCJ314NKT,-0.389290,-3.078212,4.479070,5.525538,-2.936985,8.022363,-3.088249,-3.233652,-2.785403,2.564542,...,0.533391,2.715371,6.059149,3.348130,-2.125535,-3.340732,5.016963,-2.249717,5.243899,3.879512


In [25]:
# Initialize CatBoost model
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=6, 
                           loss_function='Logloss', 
                           verbose=0)

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit model
model.fit(new_X, y)

# Get feature importances
feature_importances = model.get_feature_importance(Pool(new_X, label=y))

# Select features
selector = SelectFromModel(model, prefit=True, threshold='mean')
X_new = selector.transform(new_X)

# Get selected feature names
selected_features = new_X.columns[selector.get_support()]
print("Selected features:", selected_features)

# Create a new CatBoost model
model_selected = CatBoostClassifier(iterations=1000, 
                                    learning_rate=0.1, 
                                    depth=6, 
                                    loss_function='Logloss', 
                                    verbose=0)

# Cross-validation scores with selected features
scores = cross_val_score(model_selected, X_new, y, cv=cv, scoring='roc_auc')
print("Cross-validation AUC with selected features:", scores.mean())


/Users/zainabnazari/opt/miniconda3/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Selected features: Index(['ENSG00000131686', 'ENSG00000089041', 'ENSG00000178750',
       'ENSG00000258603', 'ENSG00000164828', 'ENSG00000173114',
       'ENSG00000223482', 'ENSG00000174332', 'ENSG00000125166',
       'ENSG00000010539', 'ENSG00000100079', 'ENSG00000237560',
       'ENSG00000163636', 'ENSG00000226237', 'ENSG00000161040',
       'ENSG00000257185', 'ENSG00000239774', 'ENSG00000182759',
       'ENSG00000235749', 'ENSG00000267024', 'ENSG00000284713',
       'ENSG00000057657', 'ENSG00000248468', 'ENSG00000166086',
       'ENSG00000278023', 'ENSG00000257156', 'ENSG00000120539',
       'ENSG00000234965', 'ENSG00000177990', 'ENSG00000116678',
       'ENSG00000122420', 'ENSG00000167851', 'ENSG00000131126',
       'ENSG00000265218', 'ENSG00000235823', 'ENSG00000163171',
       'ENSG00000247345', 'ENSG00000125356', 'ENSG00000163162',
       'ENSG00000277954', 'ENSG00000176723', 'ENSG00000242385',
       'ENSG00000152315', 'ENSG00000103154', 'ENSG00000163961',
       'ENSG000002544

# PPMI filtered

In [26]:
new_X_ = X_[common_elements]

In [27]:
# Initialize CatBoost model
model = CatBoostClassifier(iterations=1000, 
                           learning_rate=0.1, 
                           depth=6, 
                           loss_function='Logloss', 
                           verbose=0)

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit model
model.fit(new_X_, y_)

# Get feature importances
feature_importances = model.get_feature_importance(Pool(new_X_, label=y_))

# Select features
selector = SelectFromModel(model, prefit=True, threshold='mean')
X_new = selector.transform(new_X_)

# Get selected feature names
selected_features_x = new_X_.columns[selector.get_support()]
print("Selected features:", selected_features_x)

# Create a new CatBoost model
model_selected = CatBoostClassifier(iterations=1000, 
                                    learning_rate=0.1, 
                                    depth=6, 
                                    loss_function='Logloss', 
                                    verbose=0)

# Cross-validation scores with selected features
scores = cross_val_score(model_selected, X_new, y_, cv=cv, scoring='roc_auc')
print("Cross-validation AUC with selected features:", scores.mean())

/Users/zainabnazari/opt/miniconda3/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Selected features: Index(['ENSG00000185745', 'ENSG00000119922', 'ENSG00000113100',
       'ENSG00000225929', 'ENSG00000173114', 'ENSG00000223482',
       'ENSG00000134072', 'ENSG00000174332', 'ENSG00000144214',
       'ENSG00000010539', 'ENSG00000083457', 'ENSG00000171812',
       'ENSG00000139496', 'ENSG00000161040', 'ENSG00000111846',
       'ENSG00000223609', 'ENSG00000196436', 'ENSG00000265763',
       'ENSG00000163138', 'ENSG00000071967', 'ENSG00000057657',
       'ENSG00000261030', 'ENSG00000131153', 'ENSG00000230563',
       'ENSG00000248468', 'ENSG00000166086', 'ENSG00000255248',
       'ENSG00000120539', 'ENSG00000234965', 'ENSG00000177990',
       'ENSG00000269976', 'ENSG00000182389', 'ENSG00000167851',
       'ENSG00000131126', 'ENSG00000241163', 'ENSG00000247345',
       'ENSG00000276384', 'ENSG00000255079', 'ENSG00000163162',
       'ENSG00000277954', 'ENSG00000279927', 'ENSG00000267603',
       'ENSG00000164684', 'ENSG00000242385', 'ENSG00000236866',
       'ENSG000001639